In [79]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import scipy.stats as st
import requests
import json
import gmaps
import os
import warnings

# Importing Google maps API key
from api_keys import g_key

vic_wages_path = "Data files/wage by postcode.csv"
vic_wages_df = pd.read_csv(vic_wages_path, encoding='cp1252', thousands=',')

In [80]:
vic_wages_df

,"Taxation statistics 2017–18 Individuals: Selected items, by state/territory1 and postcode, 2017–18 income year2",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143,Unnamed: 144,Unnamed: 145,Unnamed: 146,Unnamed: 147
0,Notes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,State/ Territory1,Postcode,Number of individuals\nno.,Taxable income or loss3\nno.,Taxable income or loss3\n$,Tax on taxable income\nno.,Tax on taxable income\n$,Medicare levy\nno.,Medicare levy\n$,Medicare levy surcharge\nno.,...,Small business income tax offset\n$,Low income tax offset\nno.,Low income tax offset\n$,People with private health insurance\nno.,Private health insurance - your Australian Gov...,Private health insurance - your Australian Gov...,Private health insurance rebate\nno.,Private health insurance rebate\n$,Excess private health reduction or refund (reb...,Excess private health reduction or refund (reb...
2,ACT,2600,"5,962","5,930","641,711,873","5,216","188,657,453","4,578","11,922,622",172,...,"195,787","1,844","542,095","4,791","3,707","2,790,528",412,"188,764","1,793","963,746"
3,ACT,2601,"2,810","2,763","182,477,415","1,965","45,726,322","1,617","3,116,999",84,...,"47,189",842,"264,294","1,578","1,072","616,878",177,"58,759",407,"158,793"
4,ACT,2602,"20,613","20,483","1,501,063,447","17,529","360,186,331","15,218","27,101,890",706,...,"385,946","7,833","2,370,538","13,909","11,012","6,406,248","1,476","448,229","4,237","1,582,238"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2690,WA,6979,73,73,12998541,60,"4,780,235",48,"253,501",4,...,"3,882",20,"6,676",54,46,"35,003",4,962,22,"9,606"
2691,WA,6981,80,79,6407000,73,"1,590,813",67,"122,884",1,...,"4,439",29,"8,974",64,55,"31,995",10,"2,272",22,"5,326"
2692,WA,6984,56,54,4695050,47,"1,305,379",41,"89,932",0,...,"3,292",25,"7,274",41,37,"27,704",3,739,12,"6,586"
2693,WA,6985,73,72,5256510,58,"1,278,731",53,"101,269",2,...,870,21,"6,645",55,47,"24,995",6,"1,374",19,"6,226"


In [81]:
vic_wages_df.columns = vic_wages_df.iloc[1]

In [82]:
vic_wages_df

1,State/ Territory1,Postcode,Number of individuals\nno.,Taxable income or loss3\nno.,Taxable income or loss3\n$,Tax on taxable income\nno.,Tax on taxable income\n$,Medicare levy\nno.,Medicare levy\n$,Medicare levy surcharge\nno.,...,Small business income tax offset\n$,Low income tax offset\nno.,Low income tax offset\n$,People with private health insurance\nno.,Private health insurance - your Australian Government rebate received\nno.,Private health insurance - your Australian Government rebate received\n$,Private health insurance rebate\nno.,Private health insurance rebate\n$,Excess private health reduction or refund (rebate reduced)\nno.,Excess private health reduction or refund (rebate reduced)\n$
0,Notes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,State/ Territory1,Postcode,Number of individuals\nno.,Taxable income or loss3\nno.,Taxable income or loss3\n$,Tax on taxable income\nno.,Tax on taxable income\n$,Medicare levy\nno.,Medicare levy\n$,Medicare levy surcharge\nno.,...,Small business income tax offset\n$,Low income tax offset\nno.,Low income tax offset\n$,People with private health insurance\nno.,Private health insurance - your Australian Gov...,Private health insurance - your Australian Gov...,Private health insurance rebate\nno.,Private health insurance rebate\n$,Excess private health reduction or refund (reb...,Excess private health reduction or refund (reb...
2,ACT,2600,"5,962","5,930","641,711,873","5,216","188,657,453","4,578","11,922,622",172,...,"195,787","1,844","542,095","4,791","3,707","2,790,528",412,"188,764","1,793","963,746"
3,ACT,2601,"2,810","2,763","182,477,415","1,965","45,726,322","1,617","3,116,999",84,...,"47,189",842,"264,294","1,578","1,072","616,878",177,"58,759",407,"158,793"
4,ACT,2602,"20,613","20,483","1,501,063,447","17,529","360,186,331","15,218","27,101,890",706,...,"385,946","7,833","2,370,538","13,909","11,012","6,406,248","1,476","448,229","4,237","1,582,238"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2690,WA,6979,73,73,12998541,60,"4,780,235",48,"253,501",4,...,"3,882",20,"6,676",54,46,"35,003",4,962,22,"9,606"
2691,WA,6981,80,79,6407000,73,"1,590,813",67,"122,884",1,...,"4,439",29,"8,974",64,55,"31,995",10,"2,272",22,"5,326"
2692,WA,6984,56,54,4695050,47,"1,305,379",41,"89,932",0,...,"3,292",25,"7,274",41,37,"27,704",3,739,12,"6,586"
2693,WA,6985,73,72,5256510,58,"1,278,731",53,"101,269",2,...,870,21,"6,645",55,47,"24,995",6,"1,374",19,"6,226"


In [83]:
vic_wage_clean = vic_wages_df.drop(vic_wages_df.columns.difference(["State/ Territory1", "Postcode", "Number of individuals\nno.",
                                                                   "Taxable income or loss3\n$"]), axis=1)
vic_wage_clean

1,State/ Territory1,Postcode,Number of individuals\nno.,Taxable income or loss3\n$
0,Notes,NaN,NaN,NaN
1,State/ Territory1,Postcode,Number of individuals\nno.,Taxable income or loss3\n$
2,ACT,2600,"5,962","641,711,873"
3,ACT,2601,"2,810","182,477,415"
4,ACT,2602,"20,613","1,501,063,447"
...,...,...,...,...
2690,WA,6979,73,12998541
2691,WA,6981,80,6407000
2692,WA,6984,56,4695050
2693,WA,6985,73,5256510


In [84]:
vic_wage_clean.dropna()

1,State/ Territory1,Postcode,Number of individuals\nno.,Taxable income or loss3\n$
1,State/ Territory1,Postcode,Number of individuals\nno.,Taxable income or loss3\n$
2,ACT,2600,"5,962","641,711,873"
3,ACT,2601,"2,810","182,477,415"
4,ACT,2602,"20,613","1,501,063,447"
5,ACT,2603,"6,850","792,701,438"
...,...,...,...,...
2690,WA,6979,73,12998541
2691,WA,6981,80,6407000
2692,WA,6984,56,4695050
2693,WA,6985,73,5256510


In [85]:
vic_filtered = vic_wage_clean.loc[vic_wage_clean["State/ Territory1"] == "VIC"]
vic_filtered

1,State/ Territory1,Postcode,Number of individuals\nno.,Taxable income or loss3\n$
1609,VIC,3000,"24,559","1,464,326,212"
1610,VIC,3001,240,47123000
1611,VIC,3002,"3,709","687,786,435"
1612,VIC,3003,"4,520","277,885,391"
1613,VIC,3004,"8,178","976,363,798"
...,...,...,...,...
2310,VIC,3992,514,23766246
2311,VIC,3995,"5,406","228,644,019"
2312,VIC,3996,"3,286","164,139,183"
2313,VIC,8007,51,4771423


In [86]:
vic_filtered = vic_filtered.rename(columns={"State/ Territory1": "State", "Number of individuals\nno.": "No. of people", "Taxable income or loss3\n$": "Income"})
vic_filtered

1,State,Postcode,No. of people,Income
1609,VIC,3000,"24,559","1,464,326,212"
1610,VIC,3001,240,47123000
1611,VIC,3002,"3,709","687,786,435"
1612,VIC,3003,"4,520","277,885,391"
1613,VIC,3004,"8,178","976,363,798"
...,...,...,...,...
2310,VIC,3992,514,23766246
2311,VIC,3995,"5,406","228,644,019"
2312,VIC,3996,"3,286","164,139,183"
2313,VIC,8007,51,4771423


In [87]:
vic_filtered["No. of people"] = vic_filtered["No. of people"].str.replace(",","").astype(float)
vic_filtered["Income"] = vic_filtered["Income"].str.replace(",","").astype(float)

In [88]:
vic_filtered["Average income"] = vic_filtered["Income"].astype(float) / vic_filtered["No. of people"].astype(float)
vic_filtered

1,State,Postcode,No. of people,Income,Average income
1609,VIC,3000,24559.0,1.464326e+09,59624.830490
1610,VIC,3001,240.0,4.712300e+07,196345.833333
1611,VIC,3002,3709.0,6.877864e+08,185437.162308
1612,VIC,3003,4520.0,2.778854e+08,61479.068805
1613,VIC,3004,8178.0,9.763638e+08,119389.067987
...,...,...,...,...,...
2310,VIC,3992,514.0,2.376625e+07,46237.832685
2311,VIC,3995,5406.0,2.286440e+08,42294.491121
2312,VIC,3996,3286.0,1.641392e+08,49951.059951
2313,VIC,8007,51.0,4.771423e+06,93557.313725


In [89]:
vic_salary_postcode = vic_filtered.loc[:, ["Postcode", "Average income"]]

In [90]:
vic_salary_postcode

1,Postcode,Average income
1609,3000,59624.830490
1610,3001,196345.833333
1611,3002,185437.162308
1612,3003,61479.068805
1613,3004,119389.067987
...,...,...
2310,3992,46237.832685
2311,3995,42294.491121
2312,3996,49951.059951
2313,8007,93557.313725


In [91]:
vic_salary_postcode.to_csv("Data files/vic_wages_postcode.csv", encoding="utf-8", index=False, header=True)